In [134]:
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
import datetime
import funciones_nba24
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from nba_api.stats import endpoints

### En games_temporada se guarda el df que contiene las fechas y los id de juego de la temporada, entre city_a y city_b

# Estadísticas tradicionales API

### Combinaciones Games

In [213]:
a=["ATL", "BOS", "CLE", "NOP", "CHI", "DAL", "DEN", "GSW", "HOU", "LAC", "LAL", "MIA", "MIL", "MIN", "BKN", "NYK", "ORL", "IND",
"PHI", "PHX", "POR", "SAC", "SAS", "OKC", "TOR", "UTA", "MEM", "WAS", "DET", "CHA"]
combinaciones_games=[]
for i in itertools.combinations(a,2):
    combinaciones_games.append(i)
len(combinaciones_games)

435

## Obtener estadísticas tradicionales y avanzadas de la temporada

In [235]:
%%time
#DF vacío
trad = pd.DataFrame()
avanzadas = pd.DataFrame()
for i in range(len(combinaciones_games)):
    city_a = (combinaciones_games[i][0])
    city_b = (combinaciones_games[i][1])   
    #city_a="ATL"
    #city_b="BOS"
    inicio_temporada="2020-12-22"

    nba_teams = teams.get_teams()
    # Select the dictionary for the Celtics, which contains their team ID
    celtics = [team for team in nba_teams if team['abbreviation'] == city_a][0]
    celtics_id = celtics['id']

    # Query for games where the Celtics were playing
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
    # The first DataFrame of those returned is what we want.
    games = gamefinder.get_data_frames()[0]

    games["GAME_DATE"]= games ["GAME_DATE"].astype("datetime64")#Convertimos en formato fecha el game_date
    games_temporada = games[games["GAME_DATE"]>=inicio_temporada]#Escoger los partidos dentro de la temporada
    #Filtrar por la ciudad rival
    games_temporada = games_temporada[games_temporada["MATCHUP"].isin([city_a+" @ "+city_b, city_a+" vs. "+city_b])].sort_values("GAME_DATE",ascending=False)

    #reiniciar index
    games_temporada = games_temporada.reset_index()
    games_temporada.drop("index", axis=1, inplace=True)

    games_temporada["GAME_DATE"] = games_temporada["GAME_DATE"].map(lambda x: x.strftime("%Y-%m-%d"))#Cambiamos fecha a str
    fechas_partidos = games_temporada["GAME_DATE"].values

    #Guardo en id_games los id de los juegos de la temporada entre city_a y city_b
    id_games = games_temporada["GAME_ID"].values

  

    #Concatenar los df de estadísticas tradicionales de cada juego
    for i in id_games:    
        # Here we access the leagueleaders module through endpoints & assign the class to "data"
        #id="0042000207"
        data = endpoints.boxscoretraditionalv2.BoxScoreTraditionalV2(i)

        # Our "data" variable now has built in functions such as creating a dataframe for our data
        df = data.player_stats.get_data_frame()

        trad = pd.concat([trad, df])
        
   
    #Concatenar los df de estadísticas Avanzadas de cada juego
    for i in id_games:    
        # Here we access the leagueleaders module through endpoints & assign the class to "data"
   
        data = endpoints.boxscoreadvancedv2.BoxScoreAdvancedV2(i)

        # Our "data" variable now has built in functions such as creating a dataframe for our data
        df = data.player_stats.get_data_frame()
        
        avanzadas = pd.concat([avanzadas, df])

Wall time: 26min 49s


## Guardar dfs en el ordenador

In [262]:
trad.to_csv('estadisticas_tradicionales.csv', header=True, index=True)
avanzadas.to_csv('estadisticas_avanzadas.csv', header=True, index=True)

# Procesamiento de dfs trad
- Eliminar columnas que no sean necesarias.
- Eliminar NANS
- Crear los dfs de medidas de agrupación

## Eliminar columnas

In [264]:
df_tradicional = pd.read_csv("estadisticas_tradicionales.csv")

In [268]:
#Eliminar COMMENT
df_tradicional.drop("COMMENT", axis=1, inplace=True)

In [275]:
#Eliminar START_POSITION       
df_tradicional.drop("START_POSITION", axis=1, inplace=True)

In [277]:
# Eliminar las filas con nan, son jugadores que no jugaron
df_tradicional.dropna(axis=0, inplace = True) 

In [285]:
#Eliminar UNamed       
df_tradicional.drop("Unnamed: 0", axis=1, inplace=True)

In [280]:
def minutos(x):
    tiempo=[]
    for i in range(2):
        tiempo.append(int(x.split(":")[i]))
    return (tiempo[1]/60)+tiempo[0]

df_tradicional["MIN"] = df_tradicional["MIN"].apply(minutos)

### Asignar local visitante

In [305]:
local_visit=[]
for j in df_tradicional["GAME_ID"].unique():
    for i in df_tradicional[df_tradicional["GAME_ID"]==int(j)]["TEAM_CITY"]:
        if i == df_tradicional[df_tradicional["GAME_ID"]==int(j)]["TEAM_CITY"].values[0]:
            local_visit.append("V")
        else:
            local_visit.append("L")

In [308]:
df_tradicional["local_visitante"]=local_visit

### Agregar columna ganar perder

In [319]:
gan_per=[]
for j in df_tradicional["GAME_ID"].unique():
    for i in df_tradicional[df_tradicional["GAME_ID"]==int(j)]["TEAM_ABBREVIATION"]:
        if i == df_tradicional[df_tradicional["GAME_ID"]==int(j)].groupby("TEAM_ABBREVIATION").sum()["PTS"].idxmax():
            gan_per.append("G")
        else:
            gan_per.append("P")

In [321]:
df_tradicional["gan_per"]=gan_per

## Hacer df de medidas de agrupación, tradicionales

In [377]:
df_medidas_trad= pd.DataFrame()

#c1="ATL"
#c2="BOS"
for i in range(len(combinaciones_games)):
    c1 = (combinaciones_games[i][0])
    c2 = (combinaciones_games[i][1])   

    df_enfrentamientos_equipo=pd.DataFrame()
    for j in df_tradicional["GAME_ID"].unique():
        df_partido = df_tradicional[df_tradicional["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])

    media = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).mean()    
    std = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).std()
    maxim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).max()
    minim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).min()
    suma = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).sum()
    med = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).median()
    q1 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.25)
    q2 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.75)
    
    concatenacion = pd.concat([media, std, maxim, minim, suma, med, q1, q2], axis=1)
    
    df_medidas_trad = pd.concat([df_medidas_trad, concatenacion])

### Cambiar el nombre de las columnas para identificar las medidas de agrupación

In [394]:
df_medidas_trad.drop(["GAME_ID", "TEAM_ID", "PLAYER_ID"], axis=1, inplace=True)

In [420]:
df_medidas_trad.drop(["NICKNAME", "TEAM_CITY"], axis=1, inplace=True)

In [424]:
df_medidas_trad.drop(["local_visitante", "gan_per"], axis=1, inplace=True)

In [402]:
term=["_mean", "_std", "_max", "_min", "_sum", "_median", "_q25", "_q75"]
columnas_nuevas=[]
for j in term:
    b=0
    c=20
    for i in df_medidas_trad.columns[b:c]:
        columnas_nuevas.append(i+j)
    b+=20
    c+=20

In [425]:
df_medidas_trad.columns = columnas_nuevas

## Guardar df_medidas_trad

In [482]:
df_medidas_trad.to_csv('estadisticas_tradicionales_agrupadas.csv', header=True, index=True)

## Procesamiento df avanzadas

In [429]:
df_avanzadas = pd.read_csv("estadisticas_avanzadas.csv")

In [430]:
# Eliminar columnas
df_avanzadas.drop(["Unnamed: 0","COMMENT","START_POSITION" ], axis=1, inplace=True)

In [431]:
# Eliminar las filas con nan, son jugadores que no jugaron
df_avanzadas.dropna(axis=0, inplace = True) 

def minutos(x):
    tiempo=[]
    for i in range(2):
        tiempo.append(int(x.split(":")[i]))
    return (tiempo[1]/60)+tiempo[0]

df_avanzadas["MIN"] = df_avanzadas["MIN"].apply(minutos)

## Asignar local visitante

#### Es la misma lista de tradicionales

In [436]:
df_avanzadas["local_visitante"]=local_visit

## Asignar ganar perder
#### Es la misma lista de tradicionales

In [437]:
df_avanzadas["gan_per"]=gan_per

## Hacer df de medidas de agrupación, avanzadas

In [441]:
df_medidas_avan= pd.DataFrame()

#c1="ATL"
#c2="BOS"
for i in range(len(combinaciones_games)):
    c1 = (combinaciones_games[i][0])
    c2 = (combinaciones_games[i][1])   

    df_enfrentamientos_equipo=pd.DataFrame()
    for j in df_avanzadas["GAME_ID"].unique():
        df_partido = df_avanzadas[df_avanzadas["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])

    media = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).mean()    
    std = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).std()
    maxim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).max()
    minim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).min()
    suma = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).sum()
    med = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).median()
    q1 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.25)
    q2 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.75)
    
    concatenacion = pd.concat([media, std, maxim, minim, suma, med, q1, q2], axis=1)
    
    df_medidas_avan = pd.concat([df_medidas_avan, concatenacion])

### Cambiar el nombre de las columnas para identificar las medidas de agrupación

In [444]:
df_medidas_avan.drop(["GAME_ID", "TEAM_ID", "PLAYER_ID", "NICKNAME", "TEAM_CITY", "local_visitante", "gan_per"], axis=1, inplace=True)

In [455]:
term=["_mean", "_std", "_max", "_min", "_sum", "_median", "_q25", "_q75"]
columnas_nuevas=[]
for j in term:
    b=0
    c=23
    for i in df_medidas_avan.columns[b:c]:
        columnas_nuevas.append(i+j)
    b+=23
    c+=23

In [456]:
df_medidas_avan.columns = columnas_nuevas

## Guardar df_medidas_avan

In [487]:
df_medidas_avan.to_csv('estadisticas_avanzadas_agrupadas.csv', header=True, index=True)

## Guardar dfs limpios avanzadas y tradicionales

In [ ]:
df_avanzadas.to_csv('estadisticas_avanzadas.csv', header=True, index=True)
df_tradicional.to_csv('estadisticas_tradicionales.csv', header=True, index=True)